# Chapter 10 - Advanced Topics

In [1]:
import swat

In [2]:
conn = swat.CAS('server-name.mycompany.com', 5570, 'username', 'password')

## Binary vs REST Interfaces

### The Binary Interface

In [ ]:
binconn = swat.CAS('server-name.mycompany.com', 5570, protocol='cas')

### The REST Interface

In [ ]:
restconn = swat.CAS('server-name.mycompany.com', 8888, protocol='http')

## Result Processing Workflows

### The Easy Way

In [3]:
tbl = conn.loadtable('data/iris.csv', caslib='casuser').casTable

NOTE: Cloud Analytic Services made the file data/iris.csv available as table DATA.IRIS in caslib CASUSER(username).


In [4]:
out = tbl.summary()
out

,Column,Min,Max,N,NMiss,Mean,Sum,Std,StdErr,Var,USS,CSS,CV,TValue,ProbT
0,sepal_length,4.3,7.9,150.0,0.0,5.843333,876.5,0.828066,0.067611,0.685694,5223.85,102.168333,14.171126,86.425375,3.331256e-129
1,sepal_width,2.0,4.4,150.0,0.0,3.054000,458.1,0.433594,0.035403,0.188004,1427.05,28.012600,14.197587,86.264297,4.374977e-129
2,petal_length,1.0,6.9,150.0,0.0,3.758667,563.8,1.764420,0.144064,3.113179,2583.00,463.863733,46.942721,26.090198,1.994305e-57
3,petal_width,0.1,2.5,150.0,0.0,1.198667,179.8,0.763161,0.062312,0.582414,302.30,86.779733,63.667470,19.236588,3.209704e-42


### Using Response and Result Callbacks

In [5]:
def result_cb(key, value, response, connection, userdata):
    print('\n>>> RESULT %s\n' % key)
    print(value)
    return userdata

In [7]:
tbl.groupby('species').summary(resultfunc=result_cb, subset=['min', 'max'])


>>> RESULT ByGroupInfo

ByGroupInfo

      species   species_f       _key_
0      setosa      setosa      setosa
1  versicolor  versicolor  versicolor
2   virginica   virginica   virginica

>>> RESULT ByGroup1.Summary

Descriptive Statistics for DATA.IRIS

               Column  Min  Max
species                        
setosa   sepal_length  4.3  5.8
setosa    sepal_width  2.3  4.4
setosa   petal_length  1.0  1.9
setosa    petal_width  0.1  0.6

>>> RESULT ByGroup2.Summary

Descriptive Statistics for DATA.IRIS

                  Column  Min  Max
species                           
versicolor  sepal_length  4.9  7.0
versicolor   sepal_width  2.0  3.4
versicolor  petal_length  3.0  5.1
versicolor   petal_width  1.0  1.8

>>> RESULT ByGroup3.Summary

Descriptive Statistics for DATA.IRIS

                 Column  Min  Max
species                          
virginica  sepal_length  4.9  7.9
virginica   sepal_width  2.2  3.8
virginica  petal_length  4.5  6.9
virginica   petal_width  1.4  2.5


In [8]:
def response_cb(response, connection, userdata):
    print('\n>>> RESPONSE')
    for k, v in response:
        print('\n>>> RESULT %s\n' % k)
        print(v)
    return userdata

In [9]:
tbl.groupby('species').summary(responsefunc=response_cb, subset=['min', 'max'])


>>> RESPONSE

>>> RESULT ByGroupInfo

ByGroupInfo

      species   species_f       _key_
0      setosa      setosa      setosa
1  versicolor  versicolor  versicolor
2   virginica   virginica   virginica

>>> RESPONSE

>>> RESULT ByGroup1.Summary

Descriptive Statistics for DATA.IRIS

               Column  Min  Max
species                        
setosa   sepal_length  4.3  5.8
setosa    sepal_width  2.3  4.4
setosa   petal_length  1.0  1.9
setosa    petal_width  0.1  0.6

>>> RESPONSE

>>> RESULT ByGroup2.Summary

Descriptive Statistics for DATA.IRIS

                  Column  Min  Max
species                           
versicolor  sepal_length  4.9  7.0
versicolor   sepal_width  2.0  3.4
versicolor  petal_length  3.0  5.1
versicolor   petal_width  1.0  1.8

>>> RESPONSE

>>> RESULT ByGroup3.Summary

Descriptive Statistics for DATA.IRIS

                 Column  Min  Max
species                          
virginica  sepal_length  4.9  7.9
virginica   sepal_width  2.2  3.8
virginica  p

### Handling Responses from Multiple Sessions Simultaneously

In [18]:
conn1 = swat.CAS()
conn2 = swat.CAS()

In [20]:
tbl1 = conn1.loadtable('data/iris.csv', caslib='casuser').casTable
tbl2 = conn2.loadtable('data/iris.csv', caslib='casuser').casTable

NOTE: Cloud Analytic Services made the file data/iris.csv available as table DATA.IRIS in caslib CASUSER(username).
NOTE: Cloud Analytic Services made the file data/iris.csv available as table DATA.IRIS in caslib CASUSER(username).


In [27]:
conn1 = tbl1.groupby('species').invoke('summary', subset=['min', 'max'])

In [28]:
for resp in conn1:
    print('\n>>> RESPONSE')
    for k, v in resp:
        print('\n>>> RESULT %s\n' % k)
        print(v)


>>> RESPONSE

>>> RESULT ByGroupInfo

ByGroupInfo

      species   species_f       _key_
0      setosa      setosa      setosa
1  versicolor  versicolor  versicolor
2   virginica   virginica   virginica

>>> RESPONSE

>>> RESULT ByGroup1.Summary

Descriptive Statistics for DATA.IRIS

               Column  Min  Max
species                        
setosa   sepal_length  4.3  5.8
setosa    sepal_width  2.3  4.4
setosa   petal_length  1.0  1.9
setosa    petal_width  0.1  0.6

>>> RESPONSE

>>> RESULT ByGroup2.Summary

Descriptive Statistics for DATA.IRIS

                  Column  Min  Max
species                           
versicolor  sepal_length  4.9  7.0
versicolor   sepal_width  2.0  3.4
versicolor  petal_length  3.0  5.1
versicolor   petal_width  1.0  1.8

>>> RESPONSE

>>> RESULT ByGroup3.Summary

Descriptive Statistics for DATA.IRIS

                 Column  Min  Max
species                          
virginica  sepal_length  4.9  7.9
virginica   sepal_width  2.2  3.8
virginica  p

In [29]:
conn1 = tbl1.groupby('species').invoke('summary', subset=['min', 'max'])
conn2 = tbl2.groupby('species').invoke('topk', topk=1, bottomk=1)

In [30]:
for resp, c in swat.getnext(conn1, conn2):
    print('\n>>> RESPONSE')
    for k, v in resp:
        print('\n>>> RESULT %s\n' % k)
        print(v)


>>> RESPONSE

>>> RESULT ByGroupInfo

ByGroupInfo

      species   species_f       _key_
0      setosa      setosa      setosa
1  versicolor  versicolor  versicolor
2   virginica   virginica   virginica

>>> RESPONSE

>>> RESULT ByGroup1.Summary

Descriptive Statistics for DATA.IRIS

               Column  Min  Max
species                        
setosa   sepal_length  4.3  5.8
setosa    sepal_width  2.3  4.4
setosa   petal_length  1.0  1.9
setosa    petal_width  0.1  0.6

>>> RESPONSE

>>> RESULT ByGroup2.Summary

Descriptive Statistics for DATA.IRIS

                  Column  Min  Max
species                           
versicolor  sepal_length  4.9  7.0
versicolor   sepal_width  2.0  3.4
versicolor  petal_length  3.0  5.1
versicolor   petal_width  1.0  1.8

>>> RESPONSE

>>> RESULT ByGroup3.Summary

Descriptive Statistics for DATA.IRIS

                 Column  Min  Max
species                          
virginica  sepal_length  4.9  7.9
virginica   sepal_width  2.2  3.8
virginica  p

## Connecting to Existing Sessions

In [31]:
conn

CAS('server-name.mycompany.com', 5570, 'username', protocol='cas', name='py-session-1', session='2fed25d6-4946-0540-8308-73c7f75b53c6')

In [32]:
conn.listsessions()

,SessionName,UUID,State,Authentication,Userid
0,py-session-3:Fri Nov 4 10:32:16 2016,e7f753f7-4e20-aa46-ac6a-4c574cede3c5,Connected,Active Directory,username
1,py-session-1:Fri Nov 4 11:46:25 2016,2fed25d6-4946-0540-8308-73c7f75b53c6,Connected,Active Directory,username
2,py-session-2:Fri Nov 4 12:06:29 2016,6f04be18-1146-0642-8437-9e928edbf42d,Connected,Active Directory,username
3,py-session-4:Fri Nov 4 12:08:52 2016,21e426a9-20f8-ef4e-a606-cc699f8e274a,Connected,Active Directory,username
4,py-session-5:Fri Nov 4 12:08:52 2016,430d8336-2b9b-f54b-8163-86ebe476b238,Connected,Active Directory,username


In [33]:
conn2 = swat.CAS('server-name.mycompany.com', 5570, session='2fed25d6-4946-0540-8308-73c7f75b53c6')

In [34]:
conn2

CAS('server-name.mycompany.com', 5570, 'username', protocol='cas', name='py-session-6', session='2fed25d6-4946-0540-8308-73c7f75b53c6')